In [1]:
import os
import requests
import json
from pprint import pprint
from api_key import api_key
import pandas as pd

In [2]:
# Save config information
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"


states_id = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19',
             '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', 
             '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51',
             '53', '54', '55', '56']

states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 
          'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 
          'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# 01 Alabama, 02 Alaska, 04 Arizona, 05 Arkansas, 06 California, 08 Colorado, 09 Connecticut,
# 10 Delaware, 11 District of Columbia, 12 Florida, 13 Georgia, 15 Hawaii, 16 Idaho,
# 17 Illinois, 18 Indiana, 19 Iowa, 20 Kansas, 21 Kentucky, 22 Louisiana, 23 Maine, 
# 24 Maryland, 25 Massachusetts, 26 Michigan, 27 Minnesota, 28 Mississippi, 29 Missouri, 30 Montana, 
# 31 Nebraska, 32 Nevada, 33 New Hampshire, 34 New Jersey, 35 New Mexico, 36 New York, 37 North Carolina, 
# 38 North Dakota, 39 Ohio, 40 Oklahoma, 41 Oregon, 42 Pennsylvania, 44 Rhode Island, 45 South Carolina,
# 46 South Dakota, 47 Tennessee, 48 Texas, 49 Utah, 50 Vermont, 51 Virginia, 53 Washington, 54 West Virginia, 
# 55 Wisconsin, 56 Wyoming

states_data = {'states_id': states_id, 'states': states}
states_df = pd.DataFrame(states_data)
#states_df.head()

states_df_new = states_df.reset_index(drop=True)
states_short = states_df_new.set_index('states_id')

states_short.head()

,states
states_id,
01,AL
02,AK
04,AZ
05,AR
06,CA


In [3]:
# Average Weekly Wage in Private Total, all industries for All establishment sizes in Alabama -- Statewide, NSA
# Industry:      Total, all industries
# Owner:         Private
# Size:          All establishment sizes
# Type:          Average Weekly Wage

seriesid = []

for state in states_id:
    seriesid.append(f'ENU'+state+'0100040510')

#seriesid

In [4]:
# test = []
# for x in ['01','04']:
#     test.append(states_short.loc[x]

# print(test)

tryout = []
for r in range(0,4):
    tryout.append(states_df['states'][r])

tryout
                

['AL', 'AK', 'AZ', 'AR']

In [5]:
startyear = '1999'
endyear = '2018'

# Build query URL

response_all = []

for state in states_id:
    query_url = url + f'ENU'+state+'00010510' + "?" + "registrationkey=" + api_key + "&catalog=true&startyear=" + startyear + '&endyear=' + endyear + '&calculations=true&annualaverage=true&pctchange=true'
    response_json = requests.get(query_url).json()
    response_all.append(response_json)

response_all[0]

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 291,
 'message': ['No Data Available for Series ENU0100010510 Year: 1999',
  'No Data Available for Series ENU0100010510 Year: 2000',
  'Unable to get Catalog Data for series ENU0100010510'],
 'Results': {'series': [{'seriesID': 'ENU0100010510',
    'data': [{'year': '2018',
      'period': 'M13',
      'periodName': 'Annual',
      'value': '1598129',
      'footnotes': [{}],
      'calculations': {'net_changes': {'1': '-22136',
        '3': '-13336',
        '6': '-3264',
        '12': '23357'},
       'pct_changes': {'1': '-1.4', '3': '-0.8', '6': '-0.2', '12': '1.5'}}},
     {'year': '2018',
      'period': 'M12',
      'periodName': 'December',
      'value': '1620265',
      'footnotes': [{}],
      'calculations': {'net_changes': {'1': '-2515',
        '3': '17314',
        '6': '15299',
        '12': '29569'},
       'pct_changes': {'1': '-0.2', '3': '1.1', '6': '1.0', '12': '1.9'}}},
     {'year': '2018',
      'period': 'M11',


In [6]:
len(response_all)

51

In [7]:
response_all[0]['Results']['series'][0]['data'][-1]

{'year': '2001',
 'period': 'M01',
 'periodName': 'January',
 'value': '1506569',
 'footnotes': [{}]}

In [11]:
response_all[1]['Results']['series'][0]['data'][26]

{'year': '2016',
 'period': 'M13',
 'periodName': 'Annual',
 'value': '250255',
 'footnotes': [{}],
 'calculations': {'net_changes': {'1': '18186',
   '3': '6749',
   '6': '-29124',
   '12': '-5019'},
  'pct_changes': {'1': '7.8', '3': '2.8', '6': '-10.4', '12': '-2.0'}}}

In [12]:
response_all[0]['Results']['series'][0]['data'][0]['calculations']['pct_changes']

{'1': '-1.4', '3': '-0.8', '6': '-0.2', '12': '1.5'}

In [13]:
data = pd.DataFrame(columns=['state', 'year', 'value', 'pct_change'])
#print("Empty Dataframe ", data, sep='\n')

years = range(0, (2018-2002+1))
state_list = []
year_list = []
value_list = []
pct_changes_list = []

for state_index in range(len(states)):  # iterate over state indices
    for yr in years:  # iterate over years for each state index
        state_list.append(state_index)
        
        year_list.append(
            response_all[state_index]['Results']['series'][0]['data'][yr*13]['year']
        )
        value_list.append(
            response_all[state_index]['Results']['series'][0]['data'][yr*13]['value']
        )
        pct_changes_list.append(response_all[state_index]['Results']['series'][0]['data'][yr*13]['calculations']['pct_changes'])
    
print(len(state_list))
print(len(year_list))
print(len(value_list))
print(len(pct_changes_list))


867
867
867
867


In [14]:
#pct_changes_list

In [15]:
#pct_change is a list of values of each year and the last of each list is the annual percentage change

pct_changes_df = pd.DataFrame.from_dict(pct_changes_list)

pct_changes_df.head()

# The columns in pct_changes_df are '1', '3', '6' and '12'. We need to fetch the values of '12' for the 
# annual change


,1,3,6,12
0,-1.4,-0.8,-0.2,1.5
1,-1.0,-0.6,-0.1,1.2
2,-0.9,-0.4,-0.2,1.5
3,-1.7,-0.8,-0.4,1.9
4,-1.6,-0.8,-0.2,1.3


In [16]:
# df = pd.DataFrame(state_list,year_list,value_list,pct_changes_list)

values_dict = {'state_id': state_list,
               "year": year_list,
               "earnings": value_list,
              'pct_change': pct_changes_df['12']}

df = pd.DataFrame(values_dict)
df.head(20)

,state_id,year,earnings,pct_change
0,0,2018,1598129,1.5
1,0,2017,1574772,1.2
2,0,2016,1555542,1.5
3,0,2015,1532167,1.9
4,0,2014,1504320,1.3
5,0,2013,1485544,1.2
6,0,2012,1467367,1.3
7,0,2011,1449005,0.3
8,0,2010,1444072,-1.3
9,0,2009,1462992,-6.8


In [18]:
# df_state = df.groupby(['state_id'])
# df_state.

state_dfs_dictionary = {}
state_ids = set(df['state_id'])

for state in state_ids:
    state_dfs_dictionary[state] = df[df['state_id'] == state]
    
state_dfs_dictionary[0]


,state_id,year,earnings,pct_change
0,0,2018,1598129,1.5
1,0,2017,1574772,1.2
2,0,2016,1555542,1.5
3,0,2015,1532167,1.9
4,0,2014,1504320,1.3
5,0,2013,1485544,1.2
6,0,2012,1467367,1.3
7,0,2011,1449005,0.3
8,0,2010,1444072,-1.3
9,0,2009,1462992,-6.8


In [19]:
for st in range(0,51):
    filename = "csv_series/earnings_states"+str(st)+".csv"
    state_dfs_dictionary[st].to_csv(filename, index=False, header=True)
    #print(str(st)+' done')

# NOT USED (BUT PRECIOUS ANYWAY :-) )

In [ ]:
# # create a data frame for each state

# data.iloc[0]['pct_change'][0]

In [ ]:
# data = pd.DataFrame(columns=['state', 'year', 'value', 'pct_change'])
# #print("Empty Dataframe ", data, sep='\n')

# years = range(0, (2018-2001+1))

# for state_index in range(len(states)):  # iterate over state indices
#     year = []
#     value = []
#     pct_changes = []
#     for yr in years:  # iterate over years for each state index
#         year.append(
#             response_all[state_index]['Results']['series'][0]['data'][yr*13]['year']
#         )
#         value.append(
#             response_all[state_index]['Results']['series'][0]['data'][yr*13]['value']
#         )
#         pct_changes.append(response_all[state_index]['Results']['series'][0]['data'][yr*13]['calculations']['pct_changes'])
    
#     data = data.append({'state': state_index, 'year': year, 'value': value, 'pct_change': pct_changes}, ignore_index=True)

# data

In [ ]:
# # create a data frame from the years and values 
# state_list = []
# years_list = []

# for stateind in 

# for stateind in range(len(states)):  # iterate over state indices 
#     year_stateind = data.iloc[stateind]['year']
#     earnings_stateind = data.iloc[stateind]['value']
#     change_stateind = data.iloc[stateind]['pct_change'] #pct_change is a list of values of each year and the last
#                                                             # of each list is the annual percentage change

